In [61]:
import pandas as pd
import math

   
df = pd.read_csv('E:\Task scheduling project\priority.csv')
df.head(10)

,Task ID,Priority,Latency,Processing Requirements,Execution Node,Actual Execution Time (ms)
0,1,High,Low,High,Fog,50
1,2,Medium,Medium,Medium,Cloud,200
2,3,Low,High,Low,Fog,30
3,4,High,Low,High,Fog,60
4,5,Low,Medium,Medium,Cloud,150
5,6,Medium,High,Low,Cloud,120
6,7,High,Low,High,Fog,55
7,8,Low,Medium,Medium,Cloud,180
8,9,Medium,High,Low,Fog,40
9,10,High,Low,High,Fog,70


In [62]:
# Load the dataset
df = pd.read_csv('E:\\Task scheduling project\\priority.csv')

# Check the unique values in the 'Execution Node' column
print("Unique values in 'Execution Node' column:", df['Execution Node'].unique())

# Calculate the probabilities for each class
class_counts = df['Execution Node'].value_counts()
total_samples = len(df)
probabilities = class_counts / total_samples

# Ensure probabilities sum to 1
probabilities_sum = probabilities.sum()
if probabilities_sum != 1:
    probabilities /= probabilities_sum

# Calculate entropy
initial_entropy = -sum(prob * math.log2(prob) for prob in probabilities)

print(f"Entropy of the entire dataset: {initial_entropy:.4f}")

Unique values in 'Execution Node' column: ['Fog' 'Cloud']
Entropy of the entire dataset: 0.9710


In [63]:
unique_priorities = df['Priority'].unique()

for priority_value in unique_priorities:
    subset = df[df['Priority'] == priority_value]
    subset_size = len(subset)
    
    # Calculate the probabilities for each class within the subset
    class_counts = subset['Execution Node'].value_counts()
    probabilities = class_counts / subset_size
    
    # Calculate entropy for the current priority value
    entropy = -sum(prob * math.log2(prob) for prob in probabilities)
    
    print(f"Entropy of 'Execution Node' for 'Priority' {priority_value}: {entropy:.4f}")

Entropy of 'Execution Node' for 'Priority' High: -0.0000
Entropy of 'Execution Node' for 'Priority' Medium: 0.9183
Entropy of 'Execution Node' for 'Priority' Low: 0.9183


In [64]:
# Calculate the entropy of the entire dataset
def calculate_entropy(data):
    class_counts = data['Execution Node'].value_counts()
    probabilities = class_counts / len(data)
    entropy = -sum(prob * math.log2(prob) for prob in probabilities)
    return entropy

initial_entropy = calculate_entropy(df)

# Calculate the information gain for the "Priority" attribute
def calculate_information_gain(data, attribute):
    unique_values = data[attribute].unique()
    weighted_entropy_after_split = 0

    for value in unique_values:
        subset = data[data[attribute] == value]
        subset_weight = len(subset) / len(data)
        weighted_entropy_after_split += subset_weight * calculate_entropy(subset)

    information_gain = initial_entropy - weighted_entropy_after_split
    return information_gain

# Calculate information gain for the "Priority" attribute
information_gain_priority = calculate_information_gain(df, 'Priority')

print(f"Information Gain for 'Priority': {information_gain_priority:.4f}")

Information Gain for 'Priority': 0.4200


In [65]:
# Calculate information gain for the "Latency" attribute
information_gain_latency = calculate_information_gain(df, 'Latency')
information_gain_latency

0.6954618442383218

In [66]:
information_gain_processing_requirements = calculate_information_gain(df, 'Processing Requirements')
information_gain_processing_requirements

0.6954618442383218

In [67]:
print(information_gain_priority)
print(information_gain_latency)
print(information_gain_processing_requirements)

0.4199730940219749
0.6954618442383218
0.6954618442383218


In [68]:
# Choose the attribute with the highest information gain as the splitting criterion
selected_attribute = 'Latency'

# Get unique values of the selected attribute
unique_values = df[selected_attribute].unique()

# Create a dictionary to store subsets
subsets = {}

# Split the dataset into subsets based on the selected attribute
for value in unique_values:
    subsets[value] = df[df[selected_attribute] == value]

# Print the subsets
for value, subset in subsets.items():
    print(f"Subset for {selected_attribute} = {value}:\n{subset}\n")

Subset for Latency = Low:
   Task ID Priority Latency Processing Requirements Execution Node   
0        1     High     Low                    High            Fog  \
3        4     High     Low                    High            Fog   
6        7     High     Low                    High            Fog   
9       10     High     Low                    High            Fog   

   Actual Execution Time (ms)  
0                          50  
3                          60  
6                          55  
9                          70  

Subset for Latency = Medium:
   Task ID Priority Latency Processing Requirements Execution Node   
1        2   Medium  Medium                  Medium          Cloud  \
4        5      Low  Medium                  Medium          Cloud   
7        8      Low  Medium                  Medium          Cloud   

   Actual Execution Time (ms)  
1                         200  
4                         150  
7                         180  

Subset for Latency = Hi

In [70]:
def calculate_info_gain(data, attribute, target_attribute):
    # Calculate the initial entropy of the entire dataset
    initial_entropy = calculate_entropy(data[target_attribute])

    # Calculate information gain for the current attribute
    weighted_entropy_after_split = 0

    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        subset_weight = len(subset) / len(data)
        weighted_entropy_after_split += subset_weight * calculate_entropy(subset[target_attribute])

    information_gain = initial_entropy - weighted_entropy_after_split
    return information_gain

def determine_best_attribute(data, attributes, target_attribute):
    # Calculate the initial entropy of the entire dataset
    initial_entropy = calculate_entropy(data[target_attribute])

    # Initialize variables to track the best attribute and its information gain
    best_attribute = None
    max_information_gain = -1  # Initialize with a value lower than possible information gains

    # Iterate over each attribute and calculate information gain
    for attribute in attributes:
        if attribute == target_attribute:
            continue  # Skip the target attribute

        # Calculate information gain for the current attribute
        information_gain = calculate_info_gain(data, attribute, target_attribute)

        # Update the best attribute if the information gain is higher
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_attribute = attribute

    return best_attribute

def determine_class(target_values):
    # Count the occurrences of each class in target_values
    class_counts = target_values.value_counts()

    # Determine the majority class (class with the highest count)
    majority_class = class_counts.idxmax()

    return majority_class

def create_decision_tree(data, attributes, target_attribute, max_depth=None, min_samples_split=2):
    # Check stopping criteria
    if max_depth is not None and max_depth == 0:
        return {'class': determine_class(data[target_attribute])}
    if len(data) < min_samples_split:
        return {'class': determine_class(data[target_attribute])}

    # Determine the best attribute for splitting
    best_attribute = determine_best_attribute(data, attributes, target_attribute)

    # Create a decision tree node
    decision_tree = {'attribute': best_attribute, 'subsets': {}}

    # Recursively create decision tree for each subset
    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        decision_tree['subsets'][value] = create_decision_tree(subset, attributes, target_attribute,
                                                                max_depth=max_depth - 1 if max_depth is not None else None,
                                                                min_samples_split=min_samples_split)

    return decision_tree

# Example usage:
# Assuming df is the original dataframe and 'Execution Node' is the target attribute
attributes = ['Priority', 'Latency', 'Processing Requirements']
target_attribute = 'Execution Node'  # Update this if the actual column name is different
decision_tree = create_decision_tree(df, attributes, target_attribute, max_depth=3, min_samples_split=5)

# Print the decision tree (you might want to use a tree visualization library for a more readable display)
print(decision_tree)

KeyError: 'Execution Node'